In [ ]:
import pandas as pd

In [ ]:
dataset_for_generation = pd.read_csv('dataset_for_generation_documentation.csv')
dataset_for_generation.head()

,Unnamed: 0,user_code,similar_documentation
0,0,"def t_power(logu, t, self_normalized=False, na...","Simple discretization through tanh, flip bottl..."
1,1,"def update_dns_server(self, service_name, depl...",Updates a custom field belonging to this list.
2,2,"def wait_for_operation_status(self,\n r...",Receive events from the EventHub.\n\n :...
3,3,"def _resolve_distribution_names(dist_fn_args, ...",Generate features from selected columns of a d...
4,4,def _get_security_context(self):\n \n ...,Runs prior to the global main function.


In [ ]:
ideal_documentation = pd.read_csv('code_and_doc.csv')
ideal_documentation.head()

,code,docstring
0,"def t_power(logu, t, self_normalized=False, na...",The T-Power Csiszar-function in log-space.\n\n...
1,"def update_dns_server(self, service_name, depl...",Updates the ip address of a DNS server.\n\n ...
2,"def wait_for_operation_status(self,\n r...",Waits for an asynchronous operation to complet...
3,"def _resolve_distribution_names(dist_fn_args, ...",Uses arg names to resolve distribution names.
4,def _get_security_context(self):\n \n ...,Defines the security context


In [ ]:
doc_gen_hum = pd.DataFrame(columns = ['generated_documentation', 'ideal_documentation'])

In [ ]:
!pip install huggingface_hub --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.8/447.8 kB 11.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.26.3
    Uninstalling huggingface-hub-0.26.3:
      Successfully uninstalled huggingface-hub-0.26.3


In [ ]:
from huggingface_hub import InferenceClient

client = InferenceClient(api_key="Hugging Face API")

for i in range(1000):
  user_code = dataset_for_generation['user_code'][i]
  similar_documentation = dataset_for_generation['similar_documentation'][i]
  prompt = f"""Your task is to create clear and comprehensive documentation for the provided code.
The documentation should briefly explain what this code does. Write a documentation for this code:
{user_code}
This is a similar documentation: {similar_documentation}
Write a short documentation for the code. Maximum of five sentences. Try to use 1 or 2 or 3 sentences. Without an introductory sentence."""
  system_prompt = "You are a technical writing assistant specializing in generating high-quality documentation for code. You answer is just a documentation"
  messages = [
	{ "role": "user", "content": prompt},
  {"role": "system", "content": system_prompt}
]

  stream = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
	messages=messages,
	temperature=0.2,
	max_tokens=70,
	top_p=0.9,
	stream=True
)
  result = ""

  for chunk in stream:
    if "delta" in chunk.choices[0]:
        delta_content = chunk.choices[0].delta.content
        if delta_content:
            result += delta_content
  if i % 100 == 0:
    print('Iteration:', i)
  doc_gen_hum.loc[len(doc_gen_hum)] = [result, ideal_documentation['docstring'][i]]









Iteration: 0
Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900


In [ ]:
doc_gen_hum

,generated_documentation,ideal_documentation
0,This function computes the tensor power of a...,The T-Power Csiszar-function in log-space.\n\n...
1,Sure! Here is a short documentation for the ...,Updates the ip address of a DNS server.\n\n ...
2,This function waits for the completion of an...,Waits for an asynchronous operation to complet...
3,Sure! Here is a short documentation for the ...,Uses arg names to resolve distribution names.
4,This function creates a security context for...,Defines the security context
...,...,...
995,This code defines a function `operative_conf...,"Retrieve the ""operative"" configuration as a co..."
996,This function finds the location of the Java...,Find location of the java executable (helper f...
997,"Sure, here is a short documentation for the ...","For performance reasons, a lasso selection is ..."
998,This function updates an Event Hub with the ...,Updates an Event Hub.\n\n hub_name:\n ...


In [ ]:
doc_gen_hum.to_csv('generated_documentation_Llama-2-7b-chat-hf.csv')

In [ ]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 8.8 MB/s eta 0:00:00


In [ ]:
import sacrebleu


candidates = doc_gen_hum['generated_documentation'].tolist()
references = [[ref] for ref in doc_gen_hum['ideal_documentation'].tolist()]

bleu_Llama_2_7b_chat_hf = sacrebleu.corpus_bleu(candidates, references)
print(f"SacréBLEU Score: {bleu_Llama_2_7b_chat_hf.score:.2f}")

SacréBLEU Score: 46.65


In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
from nltk.translate.meteor_score import meteor_score
from nltk.tokenize import word_tokenize
import nltk

nltk.download('punkt_tab')


candidates = doc_gen_hum["generated_documentation"].apply(word_tokenize)
references = doc_gen_hum["ideal_documentation"].apply(lambda x: [word_tokenize(x)])

def calculate_final_meteor(candidates, references):
    total_score = 0
    for candidate, reference in zip(candidates, references):
        total_score += meteor_score(reference, candidate)
    return total_score / len(candidates)


final_meteor__2_7b_chat_hf = calculate_final_meteor(candidates, references)
print(f"Final METEOR Score: {final_meteor__2_7b_chat_hf:.4f}")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Final METEOR Score: 0.1755


In [ ]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9b659ef58d043cce888f76e32411a9898071d2633335e3ceda3f483dbd87ee7c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge(row):
    score = scorer.score(row["ideal_documentation"], row["generated_documentation"])
    return score['rougeL'].fmeasure

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rougeL = doc_gen_hum.apply(calculate_rouge, axis=1)


final_rougeL = rougeL.mean()
print(f"Final ROUGE-L Score: {final_rougeL:.4f}")

Final ROUGE-L Score: 0.1656


In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00


In [ ]:
from bert_score import score
candidates = doc_gen_hum["generated_documentation"][:100].tolist()
references = doc_gen_hum["ideal_documentation"][:100].tolist()

P, R, F1 = score(candidates, references, lang="en", verbose=True, model_type="roberta-base")

final_bert_score = F1.mean().item()
print(f"Final BERTScore (F1): {final_bert_score:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 318.64 seconds, 0.31 sentences/sec
Final BERTScore (F1): 0.8123


In [ ]:
!pip install transformers scipy

In [ ]:
dataset_for_generation['user_code'][13]

"def clean_perms(self):\n        \n        self.log.debug('Cleaning faulty perms')\n        sesh = self.get_session\n        pvms = (\n            sesh.query(sqla_models.PermissionView)\n            .filter(or_(\n                sqla_models.PermissionView.permission == None,  \n                sqla_models.PermissionView.view_menu == None,  \n            ))\n        )\n        deleted_count = pvms.delete()\n        sesh.commit()\n        if deleted_count:\n            self.log.info('Deleted %s faulty permissions', deleted_count)"

In [ ]:
doc_gen_hum["ideal_documentation"][13]

'FAB leaves faulty permissions that need to be cleaned up'

In [ ]:
dataset_for_generation['similar_documentation'][13]

'FAB leaves faulty permissions that need to be cleaned up'

In [ ]:
doc_gen_hum_starchat = pd.DataFrame(columns = ['generated_documentation', 'ideal_documentation'])

In [ ]:
for i in range(1000):
  user_code = dataset_for_generation['user_code'][i]
  similar_documentation = dataset_for_generation['similar_documentation'][i]
  prompt = f"""Your task is to create clear and comprehensive summarization for the provided code.
The summarization  should briefly explain what this code does. Summarization must be a connected text. Describe the functionality, not the syntax. Write a summarization for this code:
{user_code}
This is a similar documentation: {similar_documentation}
Write a summarization for this code. If it is not a long code don't write a big text. Does not use introduction sentences. Don't repeat information. Start summarization from - Function verb"""
  messages = [
	{ "role": "user", "content": prompt},
]

  stream = client.chat.completions.create(
    model="HuggingFaceH4/starchat2-15b-v0.1",
	messages=messages,
	temperature=0.2,
	max_tokens=70,
	top_p=0.9,
	stream=True
)
  result = ""

  for chunk in stream:
    if "delta" in chunk.choices[0]:
        delta_content = chunk.choices[0].delta.content
        if delta_content:
            result += delta_content
  if i % 100 == 0:
    print('Iteration:', i)
  doc_gen_hum_starchat.loc[len(doc_gen_hum_starchat)] = [result, ideal_documentation['docstring'][i]]

Iteration: 0
Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900


In [ ]:
doc_gen_hum_starchat.to_csv('generated_documentation_starchat2-15b-v0.1.csv')

In [ ]:
doc_gen_hum_starchat

,generated_documentation,ideal_documentation
1000,The provided code defines a function named `li...,It is a implementation of the Luong et al. att...
1,The provided code defines a function named `t_...,The T-Power Csiszar-function in log-space.\n\n...
2,Updates a DNS server configuration in a specif...,Updates the ip address of a DNS server.\n\n ...
3,"This code defines a function, `wait_for_operat...",Waits for an asynchronous operation to complet...
4,The code defines a function called `_resolve_d...,Uses arg names to resolve distribution names.
...,...,...
995,The code defines a method called `read_config`...,reads config
996,This code generates a formatted string represe...,"Retrieve the ""operative"" configuration as a co..."
997,"The code defines a function, _find_java(), whi...",Find location of the java executable (helper f...
998,The code defines a method called `select_lasso...,"For performance reasons, a lasso selection is ..."


In [ ]:
candidates = doc_gen_hum_starchat['generated_documentation'].tolist()
references = [[ref] for ref in doc_gen_hum_starchat['ideal_documentation'].tolist()]

bleu_starchat2 = sacrebleu.corpus_bleu(candidates, references)
print(f"SacréBLEU Score: {bleu_starchat2.score:.2f}")

SacréBLEU Score: 31.92


In [ ]:
candidates = doc_gen_hum_starchat["generated_documentation"].apply(word_tokenize)
references = doc_gen_hum_starchat["ideal_documentation"].apply(lambda x: [word_tokenize(x)])

final_meteorstarchat2 = calculate_final_meteor(candidates, references)
print(f"Final METEOR Score: {final_meteorstarchat2:.4f}")

Final METEOR Score: 0.1777


In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rougeL = doc_gen_hum_starchat.apply(calculate_rouge, axis=1)


final_rougeL_rchat2 = rougeL.mean()
print(f"Final ROUGE-L Score: {final_rougeL_rchat2:.4f}")

Final ROUGE-L Score: 0.1677


In [ ]:
candidates = doc_gen_hum_starchat["generated_documentation"][:100].tolist()  # Список сгенерированных текстов
references = doc_gen_hum_starchat["ideal_documentation"][:100].tolist()  # Список эталонных текстов

# Расчет BERTScore
P, R, F1 = score(candidates, references, lang="en", verbose=True, model_type="roberta-base")

# Итоговое значение (среднее F1)
final_bert_score__rchat2 = F1.mean().item()
print(f"Final BERTScore (F1): {final_bert_score__rchat2:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 206.13 seconds, 0.49 sentences/sec
Final BERTScore (F1): 0.8105


In [ ]:
df

,code,generated_summary
0,"def t_power(logu, t, self_normalized=False, na...",NaN
1,"def update_dns_server(self, service_name, depl...",The provided code defines a function named `t_...
2,"def wait_for_operation_status(self,\n r...",Updates a DNS server configuration in a specif...
3,"def _resolve_distribution_names(dist_fn_args, ...","This code defines a function, `wait_for_operat..."
4,def _get_security_context(self):\n \n ...,The code defines a function called `_resolve_d...
...,...,...
96,def _get_streams(self):\n \n sel...,This code defines a method 'put' within a clas...
97,"def frame(y, frame_length=2048, hop_length=512...","This code defines a method, `_get_streams`, wh..."
98,"def get_instance_group_manager(self, zone, res...",The provided code defines a function named 'fr...
99,"def execute(self, context):\n \n ...","This code defines a method, `get_instance_grou..."


In [ ]:
df.to_csv('STARCHAT.csv')

In [ ]:
doc_gen_hum_starchat

,generated_documentation,ideal_documentation
1000,The provided code defines a function named `li...,It is a implementation of the Luong et al. att...
1,The provided code defines a function named `t_...,The T-Power Csiszar-function in log-space.\n\n...
2,Updates a DNS server configuration in a specif...,Updates the ip address of a DNS server.\n\n ...
3,"This code defines a function, `wait_for_operat...",Waits for an asynchronous operation to complet...
4,The code defines a function called `_resolve_d...,Uses arg names to resolve distribution names.
...,...,...
995,The code defines a method called `read_config`...,reads config
996,This code generates a formatted string represe...,"Retrieve the ""operative"" configuration as a co..."
997,"The code defines a function, _find_java(), whi...",Find location of the java executable (helper f...
998,The code defines a method called `select_lasso...,"For performance reasons, a lasso selection is ..."


In [ ]:
doc_gen_hum_Llama3 = pd.DataFrame(columns = ['generated_documentation', 'ideal_documentation'])

In [ ]:
for i in range(1000):
  user_code = dataset_for_generation['user_code'][i]
  similar_documentation = dataset_for_generation['similar_documentation'][i]
  prompt = f"""Your task is to create clear and comprehensive summarization for the provided code.
The summarization  should briefly explain what this code does. Summarization must be a connected text. Describe the functionality, not the syntax. Write a summarization for this code:
{user_code}
This is a similar documentation: {similar_documentation}
Write a summarization for this code. If it is not a long code don't write a big text. Does not use introduction sentences. Don't repeat information. Start summarization from - Function verb"""
  messages = [
	{ "role": "user", "content": prompt},
]

  stream = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct",
	messages=messages,
	temperature=0.2,
	max_tokens=70,
	top_p=0.9,
	stream=True
)
  result = ""

  for chunk in stream:
    if "delta" in chunk.choices[0]:
        delta_content = chunk.choices[0].delta.content
        if delta_content:
            result += delta_content
  if i % 100 == 0:
    print('Iteration:', i)
  doc_gen_hum_Llama3.loc[len(doc_gen_hum_Llama3)] = [result, ideal_documentation['docstring'][i]]

Iteration: 0
Iteration: 100
Iteration: 200
Iteration: 300
Iteration: 400
Iteration: 500
Iteration: 600
Iteration: 700
Iteration: 800
Iteration: 900


In [ ]:
doc_gen_hum_Llama3.to_csv('doc_gen_hum_Llama3.csv')

In [ ]:
doc_gen_hum_Llama3

,generated_documentation,ideal_documentation
0,"Computes the tempered power of a given input, ...",The T-Power Csiszar-function in log-space.\n\n...
1,Updates a DNS server by setting its name and a...,Updates the ip address of a DNS server.\n\n ...
2,"Waits for a specific operation status, polling...",Waits for an asynchronous operation to complet...
3,Resolves distribution names by filling in miss...,Uses arg names to resolve distribution names.
4,Retrieves the security context for a Kubernete...,Defines the security context
...,...,...
995,Generates a formatted string representation of...,"Retrieve the ""operative"" configuration as a co..."
996,Locates the Java executable by checking the cu...,Find location of the java executable (helper f...
997,"Creates a selection using the Lasso method, ap...","For performance reasons, a lasso selection is ..."
998,Updates an event hub by sending a PUT request ...,Updates an Event Hub.\n\n hub_name:\n ...


In [ ]:
candidates = doc_gen_hum_Llama3['generated_documentation'].tolist()
references = [[ref] for ref in doc_gen_hum_Llama3['ideal_documentation'].tolist()]

bleu_Llama3 = sacrebleu.corpus_bleu(candidates, references)
print(f"SacréBLEU Score: {bleu_Llama3.score:.2f}")

SacréBLEU Score: 15.72


In [ ]:
candidates = doc_gen_hum_Llama3["generated_documentation"].apply(word_tokenize)
references = doc_gen_hum_Llama3["ideal_documentation"].apply(lambda x: [word_tokenize(x)])

final_meteorLlama3 = calculate_final_meteor(candidates, references)
print(f"Final METEOR Score: {final_meteorLlama3:.4f}")

Final METEOR Score: 0.1774


In [ ]:
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

rougeL = doc_gen_hum_Llama3.apply(calculate_rouge, axis=1)


final_rougeL_Llama3 = rougeL.mean()
print(f"Final ROUGE-L Score: {final_rougeL_Llama3:.4f}")

Final ROUGE-L Score: 0.1977


In [ ]:
candidates = doc_gen_hum_Llama3["generated_documentation"][:100].tolist()
references = doc_gen_hum_Llama3["ideal_documentation"][:100].tolist()

P, R, F1 = score(candidates, references, lang="en", verbose=True, model_type="roberta-base")

final_bert_score__Llama3 = F1.mean().item()
print(f"Final BERTScore (F1): {final_bert_score__Llama3:.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/4 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 120.48 seconds, 0.83 sentences/sec
Final BERTScore (F1): 0.8150
